## cmapBQ Tutorial
<br>
<div style="font-size: 10pt;line-height:20px;">
This notebook is meant to show a few examples of exploring, selecting and retrieving data available within LINCS-CMap datasets from Google BigQuery.

cmapBQ allow for targeted retrieval of relevant gene expression data from the massive resources provided by The Broad Institute and LINCS Project

### Standard Imports

In [3]:
import os

import pandas as pd

### Package imports

In [4]:
import cmapBQ.query as cmap_query
import cmapBQ.config as cmap_config

# Set up credentials
bq_client = cmap_config.get_bq_client()

<div style="font-size: 10pt;line-height:30px">
    
Alternative method of authentication:

In [5]:
#from google.cloud import bigquery
#os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'cmap-big-table-bd0276aaff22.json'
#bq_client = bigquery.Client()

<div style="font-size: 10pt;line-height:30px">
    
**[cmapPy](https://pypi.org/project/cmapPy/)** is a separate package that contains useful utilities for working with GCT(x), GRP data_types. 

In [6]:
from cmapPy.pandasGEXpress.write_gctx import write as write_gctx

<div style="font-size: 14pt;line-height:30px;font-weight:bold">
The data hosted on BigQuery is organized in a few separate tables.

<div style="font-size: 10pt;line-height:18px;font-weight:normal">
    
**compoundinfo:** Metadata for all unique compounds included in the data release. Each row contains information about 
    
**instinfo:**  <br> Sample level metadata includes information for each replicate

**siginfo:**  <br> Signature (replicate collapsed) level 5 metadata

**L1000 Level3:**  <br> Gene expression (GEX, Level 2) are normalized to invariant gene set curves and quantile normalized across each plate. Here, the data from each perturbagen treatment is referred to as a profile, experiment, or instance. Additional values for 11,350 additional genes not directly measured in the L10000 assay are inferred based on the normalized values for the 978 landmark genes.

    
**L1000 Level4:**  <br> Z-scores for each gene based on Level 3 with respect to the entire plate population. This comparison of profiles to their appropriate population control generates a list of differentially expressed genes.

**L1000 Level5:** <br> Replicate-collapsed z-score vectors based on Level 4. Replicate collapse generates one differential expression vector, which we term a signature. Connectivity analyses are performed on signatures.
    

   

In [7]:
moas = cmap_query.list_cmap_moas(bq_client)
display(moas)

,moa
0,PI3K inhibitor
1,Phosphoinositide dependent kinase inhibitor
2,Vitamin B
3,CAMP stimulant
4,EPHB3 inhibitor
...,...
701,Pyruvate kinase isozyme activator
702,Methylmalonyl CoA mutase stimulant
703,Structural glycoprotein antagonist
704,Hemozoin biocrystallization inhibitor


In [8]:
brom_inhib = cmap_query.cmap_compounds(client=bq_client, moa='Bromodomain inhibitor')

In [9]:
display(brom_inhib)

,pert_id,cmap_name,target,moa,canonical_smiles,inchi_key,compound_aliases
0,BRD-K08109215,I-BET-762,BRD4,Bromodomain inhibitor,CCNC(=O)C[C@@H]1N=C(c2ccc(Cl)cc2)c2cc(OC)ccc2-...,AAAQFGUYHFJNHI-SFHVURJKSA-N,GSK-525762A
1,BRD-K08109215,I-BET-762,BRD3,Bromodomain inhibitor,CCNC(=O)C[C@@H]1N=C(c2ccc(Cl)cc2)c2cc(OC)ccc2-...,AAAQFGUYHFJNHI-SFHVURJKSA-N,GSK-525762A
2,BRD-K08109215,I-BET-762,BRD2,Bromodomain inhibitor,CCNC(=O)C[C@@H]1N=C(c2ccc(Cl)cc2)c2cc(OC)ccc2-...,AAAQFGUYHFJNHI-SFHVURJKSA-N,GSK-525762A


<div style="font-size: 12pt;line-height:30px">

In another example, we can query the dataset to look for all available gene targets using list_cmap_targets(). From this list of targets, we will see if our desired target **'CDK1'** is witin the database. 

In [10]:
targets = cmap_query.list_cmap_targets(bq_client)
display(targets)

,target
0,NR1I3
1,PIK3CG
2,GSK3B
3,UGCG
4,PGR
...,...
934,PPAT
935,ERBB3
936,HRH4
937,TLR4


In [11]:
'CDK1' in targets.target.unique()

True

<div style="font-size: 12pt;line-height:30px">

If the desired target is in the database, we can then query the compound table to get information about what compounds affect the CDK1 target. 

In [12]:
CDK1_cpinfo = cmap_query.cmap_compounds(client=bq_client, target='CDK1', verbose=True)
display(CDK1_cpinfo)

Table: 
 cmap-big-table.broad_cmap_lincs_data.compoundinfo
Query:
 SELECT * FROM cmap-big-table.broad_cmap_lincs_data.compoundinfo WHERE target in UNNEST(['CDK1'])


,pert_id,cmap_name,target,moa,canonical_smiles,inchi_key,compound_aliases
0,BRD-K87909389,alvocidib,CDK1,CDK inhibitor,CN1CC[C@@H]([C@H](O)C1)c1c(O)cc(O)c2c1oc(cc2=O...,BIIVYFLTOXDAOV-YVEFUNNKSA-N,None
1,BRD-K17894950,indirubin,CDK1,CDK inhibitor,O=C1Nc2ccccc2C1=C1Nc2ccccc2C1=O,CRDNMYFJWFXOCH-YPKPFQOOSA-N,None
2,BRD-K17894950,indirubin,CDK1,GSK-3 inhibitor,O=C1Nc2ccccc2C1=C1Nc2ccccc2C1=O,CRDNMYFJWFXOCH-YPKPFQOOSA-N,None
3,BRD-K31268420,NSC-693868,CDK1,GSK-3 inhibitor,Nc1nnc2[nH]c3ccccc3nc12,DWHVZCLBMTZRQM-UHFFFAOYSA-N,None
4,BRD-K87932577,NSC-693868,CDK1,GSK-3 inhibitor,Nc1n[nH]c2nc3ccccc3nc12,DWHVZCLBMTZRQM-UHFFFAOYSA-N,None
5,BRD-K87932577,NSC-693868,CDK1,CDK inhibitor,Nc1n[nH]c2nc3ccccc3nc12,DWHVZCLBMTZRQM-UHFFFAOYSA-N,None
6,BRD-K31268420,NSC-693868,CDK1,CDK inhibitor,Nc1nnc2[nH]c3ccccc3nc12,DWHVZCLBMTZRQM-UHFFFAOYSA-N,None
7,BRD-K82731415,olomoucine,CDK1,CDK inhibitor,Cn1cnc2c(NCc3ccccc3)nc(NCCO)nc12,GTVPOLSIJWJJNY-UHFFFAOYSA-N,None
8,BRD-K53959060,indirubin,CDK1,GSK-3 inhibitor,ON=C1C(=C2C(=O)Nc3ccccc23)Nc4ccccc14,HBDSHCUSXQATPO-UHFFFAOYSA-N,None
9,BRD-K53959060,indirubin,CDK1,CDK inhibitor,ON=C1C(=C2C(=O)Nc3ccccc23)Nc4ccccc14,HBDSHCUSXQATPO-UHFFFAOYSA-N,None


<div style="font-size: 12pt;line-height:30px">

Lets take the first 10 compounds and see how many signatures are available for those compounds. We can pass a list of compounds to the **cmap_sig function**, which then queries the dataset for compounds that match.

In [13]:
CDK1_cps = CDK1_cpinfo.pert_id.unique()

In [14]:
CDK1_cps[1:10]

array(['BRD-K17894950', 'BRD-K31268420', 'BRD-K87932577', 'BRD-K82731415',
       'BRD-K53959060', 'BRD-K64800655', 'BRD-K79090631', 'BRD-K11636097',
       'BRD-K13390322'], dtype=object)

In [15]:
CDK1_siginfo = cmap_query.cmap_sig(bq_client, pert_id=list(CDK1_cps), verbose=True)

Table: 
 cmap-big-table.cmap_lincs_public_views.siginfo
Query:
 SELECT * FROM cmap-big-table.cmap_lincs_public_views.siginfo WHERE pert_id in UNNEST(['BRD-K87909389', 'BRD-K17894950', 'BRD-K31268420', 'BRD-K87932577', 'BRD-K82731415', 'BRD-K53959060', 'BRD-K64800655', 'BRD-K79090631', 'BRD-K11636097', 'BRD-K13390322', 'BRD-K13662825', 'BRD-K50836978', 'BRD-K35687265', 'BRD-K37312348', 'BRD-K07762753', 'BRD-K60997853', 'BRD-K62814476', 'BRD-K14560436', 'BRD-K71726959', 'BRD-K78373679', 'BRD-K03273112', 'BRD-K45293975'])


In [16]:
CDK1_siginfo

,bead_batch,nearest_dose,pert_dose,pert_dose_unit,pert_idose,pert_itime,pert_time,pert_time_unit,cell_mfc_name,pert_mfc_id,...,pert_iname,pert_type,cell_iname,id,det_wells,det_plates,distill_ids,distil_ids,build_name,project_code
0,b33,10.00,10.00000,uM,None,3 h,3.0,h,MCF10A.WT,BRD-K14560436,...,TG-02,trt_cp,MCF10A,NaN,A16,LCP001_MCF10A.WT_3H_X1.A2_B33|LCP001_MCF10A.WT...,None,LCP001_MCF10A.WT_3H_X1.A2_B33:A16|LCP001_MCF10...,None,LCP
1,b33,1.11,1.11111,uM,None,3 h,3.0,h,MCF10A.WT,BRD-K14560436,...,TG-02,trt_cp,MCF10A,NaN,A18,LCP001_MCF10A.WT_3H_X1.A2_B33|LCP001_MCF10A.WT...,None,LCP001_MCF10A.WT_3H_X1.A2_B33:A18|LCP001_MCF10...,None,LCP
2,b33,3.33,3.33333,uM,None,3 h,3.0,h,MCF10A.WT,BRD-K14560436,...,TG-02,trt_cp,MCF10A,NaN,A17,LCP001_MCF10A.WT_3H_X1.A2_B33|LCP001_MCF10A.WT...,None,LCP001_MCF10A.WT_3H_X1.A2_B33:A17|LCP001_MCF10...,None,LCP
3,b33,1.11,1.11111,uM,None,6 h,6.0,h,MCF10A.WT,BRD-K14560436,...,TG-02,trt_cp,MCF10A,NaN,A18,LCP001_MCF10A.WT_6H_X1.A2_B33|LCP001_MCF10A.WT...,None,LCP001_MCF10A.WT_6H_X1.A2_B33:A18|LCP001_MCF10...,None,LCP
4,b33,3.33,3.33333,uM,None,6 h,6.0,h,MCF10A.WT,BRD-K14560436,...,TG-02,trt_cp,MCF10A,NaN,A17,LCP001_MCF10A.WT_6H_X1.A2_B33|LCP001_MCF10A.WT...,None,LCP001_MCF10A.WT_6H_X1.A2_B33:A17|LCP001_MCF10...,None,LCP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4255,b42,3.33,3.33000,uM,None,24 h,24.0,h,HAP1.PRKACA,BRD-K45293975,...,7-hydroxystaurosporine,trt_cp,HAP1,NaN,A13,MOAR018_HAP1.PRKACA_24H_X1_B42|MOAR018_HAP1.PR...,None,MOAR018_HAP1.PRKACA_24H_X1_B42:A13|MOAR018_HAP...,None,MOAR
4256,b42,0.66,0.67000,uM,None,24 h,24.0,h,HAP1.PRKACA,BRD-K45293975,...,7-hydroxystaurosporine,trt_cp,HAP1,NaN,A14,MOAR018_HAP1.PRKACA_24H_X1_B42|MOAR018_HAP1.PR...,None,MOAR018_HAP1.PRKACA_24H_X1_B42:A14|MOAR018_HAP...,None,MOAR
4257,b33,1.11,1.11111,uM,None,24 h,24.0,h,MCF10A.PIK3CA.HH,BRD-K45293975,...,7-hydroxystaurosporine,trt_cp,MCF10A,NaN,B24,LCP001_MCF10A.PIK3CA.HH_24H_X1_B33|LCP001_MCF1...,None,LCP001_MCF10A.PIK3CA.HH_24H_X1_B33:B24|LCP001_...,None,LCP
4258,b33,3.33,3.33333,uM,None,24 h,24.0,h,MCF10A.PIK3CA.HH,BRD-K45293975,...,7-hydroxystaurosporine,trt_cp,MCF10A,NaN,B23,LCP001_MCF10A.PIK3CA.HH_24H_X1_B33|LCP001_MCF1...,None,LCP001_MCF10A.PIK3CA.HH_24H_X1_B33:B23|LCP001_...,None,LCP


<div style="font-size: 12pt;line-height:30px">


The siginfo file provides information on the conditions for each experiment such as compound, dose, timepoint, cell line, and more.

The table also includes information regarding the signal strength and replicate correlation of the compound. The `distil_tas` contains the signatures **Transcriptional Activity Score (TAS)** which is an aggregate measure of strength and reproducibilty.  [More information about signature quality metrics can be found on Connectopedia](https://clue.io/connectopedia/signature_quality_metrics)

In [17]:
print("Time points:\n {}".format(CDK1_siginfo.pert_itime.unique()))
print("Doses: \n {}".format(CDK1_siginfo.pert_idose.unique()))

Time points:
 ['3 h' '6 h' '12 h' '24 h' '48 h' '4 h']
Doses: 
 [None]


In [18]:
filtered_CDK1 = CDK1_siginfo.loc[
    (CDK1_siginfo.pert_dose == 10 ) & 
    (CDK1_siginfo.pert_itime == '24 h' )
]
display(filtered_CDK1[0:10])

,bead_batch,nearest_dose,pert_dose,pert_dose_unit,pert_idose,pert_itime,pert_time,pert_time_unit,cell_mfc_name,pert_mfc_id,...,pert_iname,pert_type,cell_iname,id,det_wells,det_plates,distill_ids,distil_ids,build_name,project_code
9,b33,10.0,10.0,uM,None,24 h,24.0,h,MCF10A.WT,BRD-K14560436,...,TG-02,trt_cp,MCF10A,NaN,A16,LCP001_MCF10A.WT_24H_X1.A2_B33|LCP001_MCF10A.W...,None,LCP001_MCF10A.WT_24H_X1.A2_B33:A16|LCP001_MCF1...,None,LCP
13,b33,10.0,10.0,uM,None,24 h,24.0,h,MCF10A.PIK3CA.HH,BRD-K14560436,...,TG-02,trt_cp,MCF10A,NaN,A16,LCP001_MCF10A.PIK3CA.HH_24H_X1_B33|LCP001_MCF1...,None,LCP001_MCF10A.PIK3CA.HH_24H_X1_B33:A16|LCP001_...,None,LCP
24,b19,10.0,10.0,uM,None,24 h,24.0,h,A549,BRD-K13390322,...,AT-7519,trt_cp,A549,NaN,K01,LJP006_A549_24H_X3_B19,None,LJP006_A549_24H_X3_B19:K01,None,LJP
30,b19,10.0,10.0,uM,None,24 h,24.0,h,HEPG2,BRD-K13390322,...,AT-7519,trt_cp,HEPG2,NaN,K01,LJP006_HEPG2_24H_X1_B19,None,LJP006_HEPG2_24H_X1_B19:K01,None,LJP
31,b17,10.0,10.0,uM,None,24 h,24.0,h,SKBR3,BRD-K13390322,...,AT-7519,trt_cp,SKBR3,NaN,K01,LJP006_SKBR3_24H_X2_B17,None,LJP006_SKBR3_24H_X2_B17:K01,None,LJP
38,b19,10.0,10.0,uM,None,24 h,24.0,h,HCC515,BRD-K13390322,...,AT-7519,trt_cp,HCC515,NaN,K01,LJP006_HCC515_24H_X3_B19,None,LJP006_HCC515_24H_X3_B19:K01,None,LJP
39,b17,10.0,10.0,uM,None,24 h,24.0,h,HS578T,BRD-K13390322,...,AT-7519,trt_cp,HS578T,NaN,K01,LJP006_HS578T_24H_X2_B17,None,LJP006_HS578T_24H_X2_B17:K01,None,LJP
41,b32,10.0,10.0,uM,None,24 h,24.0,h,THP1,BRD-K13390322,...,AT-7519,trt_cp,THP1,NaN,H19,REP.A010_THP1_24H_X3_B32,None,REP.A010_THP1_24H_X3_B32:H19,None,REP
53,b17,10.0,10.0,uM,None,24 h,24.0,h,MDAMB231,BRD-K13390322,...,AT-7519,trt_cp,MDAMB231,NaN,K01,LJP006_MDAMB231_24H_X2_B17,None,LJP006_MDAMB231_24H_X2_B17:K01,None,LJP
63,b5,10.0,10.0,uM,None,24 h,24.0,h,VCAP,BRD-K13390322,...,AT-7519,trt_cp,VCAP,NaN,K04,CPC013_VCAP_24H_X3_B5_DUO52HI53LO,None,CPC013_VCAP_24H_X3_B5_DUO52HI53LO:K04,None,CPC


<div style="font-size: 12pt;line-height:30px">

From this table if we want the numerical data, we can extract the sig_ids and use them to query Level 5 database

In [19]:
CDK1_sig_ids = list(filtered_CDK1.sig_id.unique())

In [20]:
CDK1_data = cmap_query.cmap_matrix(bq_client, data_level='level5', cid=list(CDK1_siginfo.sig_id.unique()[0:1000]))

Running query ... (1/1)
Pivoting Dataframes to GCT objects
Pivoting... (1/1)
Complete


In [21]:
CDK1_data.data_df

cid,CPC013_A375_6H:BRD-K13390322-001-01-4:10,CPC013_A375_6H:BRD-K35687265-236-01-8:10,CPC013_A549_24H:BRD-K13390322-001-01-4:10,CPC013_A549_24H:BRD-K35687265-236-01-8:10,CPC013_A549_6H:BRD-K13390322-001-01-4:10,CPC013_A549_6H:BRD-K35687265-236-01-8:10,CPC013_ASC_24H:BRD-K13390322-001-01-4:10,CPC013_ASC_24H:BRD-K35687265-236-01-8:10,CPC013_HA1E_6H:BRD-K13390322-001-01-4:10,CPC013_HA1E_6H:BRD-K35687265-236-01-8:10,...,REP.B010_THP1_24H:H20,REP.B010_THP1_24H:H21,REP.B010_THP1_24H:H22,REP.B010_THP1_24H:H23,REP.B010_THP1_24H:H24,REP.B010_YAPC_24H:H20,REP.B010_YAPC_24H:H21,REP.B010_YAPC_24H:H22,REP.B010_YAPC_24H:H23,REP.B010_YAPC_24H:H24
rid,,,,,,,,,,,,,,,,,,,,,
10,1.185874,-0.755140,2.198585,0.142736,0.65130,1.04160,1.119104,1.244991,1.342418,-0.24075,...,-0.5165,0.1779,0.4730,-0.6473,2.3583,-0.52375,-0.69800,0.78370,-0.01310,0.93090
100,0.696834,0.422637,1.202844,0.205725,-0.24250,0.04610,2.054602,0.284038,0.672675,-0.08910,...,-1.4070,-0.0255,-1.3465,0.0862,-0.5013,-1.18470,0.44690,-0.46790,0.14660,-0.66150
1000,-1.083696,0.472077,-0.971127,-1.291404,0.86940,2.39490,-0.554291,-0.814638,-0.728171,0.90575,...,0.4894,-0.8271,-0.3009,0.5749,-0.3269,-1.02605,-0.44180,-0.60815,-0.79565,0.96565
10000,0.222204,1.264480,0.643960,1.651972,0.56995,1.64155,1.611529,-0.252383,-0.316395,0.72670,...,1.1318,-0.4356,-1.5301,-1.7060,-0.4083,-1.03325,-0.33220,-0.74380,0.04160,-0.72080
10001,-3.254283,-0.499116,-2.736520,0.238186,-2.54485,1.84870,-1.354021,-1.284102,-2.195163,-0.04860,...,-0.6464,0.5009,1.8189,-0.4062,0.5950,1.31095,0.71035,1.19365,-0.65685,-0.12895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9990,0.176370,1.010904,1.712255,1.358158,0.91145,-0.79630,1.329882,1.829830,0.302232,-0.12655,...,-0.7778,-0.7677,1.5271,-0.2404,-1.1305,-0.62705,-0.38080,0.98915,1.74755,1.08035
9991,-0.606921,-0.430509,1.217269,0.161526,0.65505,0.65290,0.299556,1.488463,0.883985,1.96890,...,0.6310,-0.5325,-0.5431,0.7035,0.3294,0.40080,1.50040,0.53195,-1.02535,-1.03965
9992,3.296041,-0.034924,2.231316,1.897164,1.09635,-1.42030,1.546041,1.177567,0.784149,-1.65690,...,-0.3122,-0.1523,1.3027,-0.4895,0.1480,-0.93155,-0.88985,-0.25435,1.26340,0.12340


In [22]:
#write_gctx(CDK1_data, filename)